In [ ]:
!pip install flask_cors
!pip install pillow

import time
import cv2
import dlib
import math
import numpy as np
from scipy.spatial import Delaunay
from io import BytesIO
import io
from PIL import Image
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import base64
import os

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static'
CORS(app)

face_detector = dlib.get_frontal_face_detector()
predictor_path = 'model/shape_predictor_68_face_landmarks.dat'
shape_predictor = dlib.shape_predictor(predictor_path)



@app.route('/upload_and_receive', methods=['GET','POST'])
def upload_and_receive():
    try:
        if 'image' not in request.files:
            return jsonify({'error': 'No image provided'}), 400
        
        uploaded_image = request.files['image']
        image_data = uploaded_image.read()
        input_image = Image.open(BytesIO(image_data))

        output_image = transform_image(input_image)

        static_folder = 'static'
        output_image_path = os.path.join(static_folder, 'transformed_image.jpg')

        os.makedirs(static_folder, exist_ok=True)

        cv2.imwrite(output_image_path, output_image)

        return send_file(output_image_path, mimetype='image/jpeg')

    except Exception as e:
        app.logger.error(f'Error in /upload_and_receive: {str(e)}')
        return jsonify({'error': 'Internal Server Error'}), 500
    

def transform_image(customer_img):
        face_detector = dlib.get_frontal_face_detector()

        predictor_path = 'model/shape_predictor_68_face_landmarks.dat'
        shape_predictor = dlib.shape_predictor(predictor_path)

        addict_img_path = 'resources/addictor2.png'

        face_thin_alpha = 1 
        cut_image = True
        exposure_alpha = 1 
        customer_img = np.array(customer_img)
        addict_img = cv2.imread(addict_img_path)

        if addict_img is not None:
            # 이미지를 BGR에서 RGB로 변환
            addict_img = cv2.cvtColor(addict_img, cv2.COLOR_BGR2RGB)
        else:
            # 이미지를 읽지 못한 경우 오류 처리
            raise ValueError('Failed to read the image.')

        assert customer_img is not None, 'empty image'
        assert addict_img is not None, 'empty image'

        now = time.time()
        if cut_image:
            customer_img = detect_face_and_cut(customer_img)


        print("5")

        customer_thin_image = face_thin(customer_img, alpha=face_thin_alpha)
        customer_thin_image_2 = face_thin(customer_thin_image,
                                          alpha=face_thin_alpha)

        print("6")

        merge_image = face_morph(customer_thin_image,
                                 addict_img,
                                 alpha=0.2,
                                 exposure_alpha=exposure_alpha)
        merge_image_2 = face_morph(customer_thin_image,
                                   addict_img,
                                   alpha=0.4,
                                   exposure_alpha=exposure_alpha)
        merge_image_3 = face_morph(customer_thin_image_2,
                                   addict_img,
                                   alpha=0.6,
                                   exposure_alpha=exposure_alpha)
        merge_image_4 = face_morph(customer_thin_image_2,
                                   addict_img,
                                   alpha=0.8,
                                   exposure_alpha=exposure_alpha)
        
        output_image = merge_image_3
        output_image = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)
        
        return output_image

def detect_face_and_cut(img: np.ndarray) -> np.ndarray:
 
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    rects = face_detector(img_gray, 0)


    if len(rects) == 1:
        rect = rects[0]
  
        height = rect.bottom() - rect.top()
        width = rect.right() - rect.left()
       
        new_top, new_left = int(max(rect.top() - height * 0.5,
                                    0)), int(max(rect.left() - width * 0.5, 0))
        new_bottom, new_right = int(
            min(rect.bottom() + height * 0.5, img.shape[0])), int(
                min(rect.right() + width * 0.5, img.shape[1]))


        img_rect = img[new_top:new_bottom, new_left:new_right]
        return img_rect

    raise RuntimeError(
        f'There are {len(rects)} faces in your image, expecting only 1')


def get_face_68_landmarks(img: np.ndarray) -> np.ndarray:

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = face_detector(img_gray, 1)
    return np.array([[p.x, p.y]
                     for p in shape_predictor(img, rects[0]).parts()])


def local_translation_warp(img: np.ndarray, center_point: np.ndarray,
                           mouse_point: np.ndarray,
                           radius: float) -> np.ndarray:
 
    img_copy = img.copy()

    height, width, _ = img.shape
  
    left, right = int(max(center_point[0] - radius, 0)), \
        int(min(center_point[0] + radius, width))
    bottom, high = int(max(center_point[1] - radius, 0)), \
        int(min(center_point[1] + radius, height))


    mc_distance_pow = (mouse_point[0] - center_point[0])**2 + (
        mouse_point[1] - center_point[1])**2
    radius_pow = radius**2

    for i in range(left, right):
        for j in range(bottom, high):
           
            distance = (i - center_point[0])**2 + (j - center_point[1])**2

   
            if distance < radius_pow:
               
                tmp = ((radius_pow - distance) /
                       (radius_pow - distance + mc_distance_pow))**2
                x = i - tmp * (mouse_point[0] - center_point[0])
                y = j - tmp * (mouse_point[1] - center_point[1])

              
                value = BilinearInsert(img, x, y)
                
                img_copy[j, i] = value

    return img_copy


def BilinearInsert(image: np.ndarray, x: float, y: float) -> np.ndarray:
 
    _, _, c = image.shape
    if c == 3:
        x1, y1 = int(x), int(y)
        x2, y2 = x1 + 1, y1 + 1
       
        part1 = image[y1, x1] * (x2 - x) * (y2 - y)
        part2 = image[y1, x2] * (x - x1) * (y2 - y)
        part3 = image[y2, x1] * (x2 - x) * (y - y1)
        part4 = image[y2, x2] * (x - x1) * (y - y1)
        insert_value = part1 + part2 + part3 + part4
        return insert_value.astype(np.int8)

    raise RuntimeError('BilinearInsert')


def face_thin(img: np.ndarray, alpha: float = 1) -> np.ndarray:

    landmarks = get_face_68_landmarks(img)

    left_landmark = landmarks[3] 
    left_landmark_down = landmarks[5] 
    right_landmark = landmarks[13]
    right_landmark_down = landmarks[11]
    middle_landmark = landmarks[30] 

  
    r_left = math.sqrt((left_landmark[0] - left_landmark_down[0])**2 +
                       (left_landmark[1] - left_landmark_down[1])**2)

  
    r_right = math.sqrt((right_landmark_down[0] - right_landmark[0])**2 +
                        (right_landmark_down[1] - right_landmark[1])**2)

  
    customer_thin_image = local_translation_warp(img, left_landmark,
                                                 middle_landmark,
                                                 r_left * alpha)
    
    customer_thin_image = local_translation_warp(customer_thin_image,
                                                 right_landmark,
                                                 middle_landmark,
                                                 r_right * alpha)

    return customer_thin_image


def landmarks_add_8_points(image: np.ndarray,
                           points: np.ndarray) -> np.ndarray:
  
    x, y = image.shape[1] - 1, image.shape[0] - 1
    points = points.tolist()
    points.append([0, 0])
    points.append([x // 2, 0])
    points.append([x, 0])
    points.append([x, y // 2])
    points.append([x, y])
    points.append([x // 2, y])
    points.append([0, y])
    points.append([0, y // 2])
    return np.array(points)


def affine_transform(input_image: np.ndarray, input_triangle: list,
                     output_triangle: list, size: tuple) -> np.ndarray:
   
    affine_matrix = cv2.getAffineTransform(np.float32(input_triangle),
                                           np.float32(output_triangle))

    result = cv2.warpAffine(input_image,
                            affine_matrix, (size[0], size[1]),
                            None,
                            flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_REFLECT_101)
    return result


def morph_one_triangle(bottom_img: np.ndarray, mask_img: np.ndarray,
                       morph_img: np.ndarray, bottom_tri: list, mask_tri: list,
                       morph_tri: list, alpha: float) -> None:
 
    bottom_tri_rect, mask_tri_rect = cv2.boundingRect(np.float32(
        [bottom_tri])), cv2.boundingRect(np.float32([mask_tri]))
    morph_tri_rect = cv2.boundingRect(np.float32([morph_tri]))

   
    bottom_tri_deal, mask_tri_deal, morph_tri_deal = [], [], []

 
    for i in range(3):
        morph_tri_deal.append(((morph_tri[i][0] - morph_tri_rect[0]),
                               (morph_tri[i][1] - morph_tri_rect[1])))
        bottom_tri_deal.append(((bottom_tri[i][0] - bottom_tri_rect[0]),
                                (bottom_tri[i][1] - bottom_tri_rect[1])))
        mask_tri_deal.append(((mask_tri[i][0] - mask_tri_rect[0]),
                              (mask_tri[i][1] - mask_tri_rect[1])))

   
    bottom_img_rect = bottom_img[bottom_tri_rect[1]:bottom_tri_rect[1] +
                                 bottom_tri_rect[3],
                                 bottom_tri_rect[0]:bottom_tri_rect[0] +
                                 bottom_tri_rect[2]]
    mask_img_rect = mask_img[mask_tri_rect[1]:mask_tri_rect[1] +
                             mask_tri_rect[3],
                             mask_tri_rect[0]:mask_tri_rect[0] +
                             mask_tri_rect[2]]

   
    size = (morph_tri_rect[2], morph_tri_rect[3])
    bottom_affine_img = affine_transform(bottom_img_rect, bottom_tri_deal,
                                         morph_tri_deal, size)
    mask_affine_img = affine_transform(mask_img_rect, mask_tri_deal,
                                       morph_tri_deal, size)

  
    img_rect = (1.0 - alpha) * bottom_affine_img + alpha * mask_affine_img

 
    mask = np.zeros((morph_tri_rect[3], morph_tri_rect[2], 3),
                    dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(morph_tri_deal), (1, 1, 1), 16)

   
    morph_img[morph_tri_rect[1]:morph_tri_rect[1] + morph_tri_rect[3],
              morph_tri_rect[0]:morph_tri_rect[0] + morph_tri_rect[2]] = \
        morph_img[morph_tri_rect[1]:morph_tri_rect[1] + morph_tri_rect[3],
                  morph_tri_rect[0]:morph_tri_rect[0] + morph_tri_rect[2]] * \
        (1 - mask) + img_rect * mask


def triangle_face_morph(bottom_img: np.ndarray,
                        mask_img: np.ndarray,
                        landmarks_bottom: np.ndarray,
                        landmarks_mask: np.ndarray,
                        alpha: float = 0.5) -> np.ndarray:
  
    landmarks_bottom = landmarks_add_8_points(bottom_img, landmarks_bottom)
    landmarks_mask = landmarks_add_8_points(mask_img, landmarks_mask)

  
    landmarks_morph = (1 - alpha) * landmarks_bottom + alpha * landmarks_mask

  
    bottom_img, mask_img = np.float32(bottom_img), np.float32(mask_img)

    morph_img = np.zeros(bottom_img.shape, dtype=bottom_img.dtype)

   
    triangles = Delaunay(landmarks_morph).simplices

   
    for triangle in triangles:
        point_1, point_2, point_3 = triangle[0], triangle[1], triangle[2]

    
        bottom_triangle = [
            landmarks_bottom[point_1], landmarks_bottom[point_2],
            landmarks_bottom[point_3]
        ]
        mask_triangle = [
            landmarks_mask[point_1], landmarks_mask[point_2],
            landmarks_mask[point_3]
        ]
        morph_triangle = [
            landmarks_morph[point_1], landmarks_morph[point_2],
            landmarks_morph[point_3]
        ]

        morph_one_triangle(bottom_img, mask_img, morph_img, bottom_triangle,
                           mask_triangle, morph_triangle, alpha)

    return np.uint8(morph_img)


def merge_img(bottom_img: np.ndarray,
              morph_img: np.ndarray,
              landmarks_bottom: np.ndarray,
              exposure_alpha: float = 1.1) -> np.ndarray:
 
    kernel = int(0.4 * np.linalg.norm(
        np.mean(landmarks_bottom[36:42], axis=0) -
        np.mean(landmarks_bottom[42:48], axis=0)))
    if kernel % 2 == 0:
        kernel += 1

 
    bottom_img_blur = cv2.GaussianBlur(bottom_img, (kernel, kernel), 0)
    morph_img_blur = cv2.GaussianBlur(morph_img, (kernel, kernel), 0)


    morph_img_blur += (128 * (morph_img_blur <= 1.0)).astype(np.uint8)
    morph_img = np.uint8(
        morph_img.astype(np.float32) * bottom_img_blur.astype(np.float32) /
        morph_img_blur.astype(np.float32) / exposure_alpha)


    face_mask = np.zeros(bottom_img.shape, dtype=bottom_img.dtype)
    cover = list(range(0, 27)) 
    cv2.fillConvexPoly(face_mask, cv2.convexHull(landmarks_bottom[cover]),
                       (255, 255, 255))
    r = cv2.boundingRect(landmarks_bottom)  
    center = (r[0] + int(r[2] / 2), r[1] + int(r[3] / 2)) 
    face_mask = cv2.blur(face_mask, (15, 10), center)

  
    result = cv2.seamlessClone(morph_img, bottom_img, face_mask, center,
                               cv2.NORMAL_CLONE)

    return result


def affine_transform_and_change_shape(img: np.ndarray,
                                      landmarks_target: np.ndarray,
                                      shape_target: tuple) -> np.ndarray:
  
   
    landmarks_img = np.matrix(get_face_68_landmarks(img)).astype(np.float64)
    landmarks_target = np.matrix(landmarks_target).astype(np.float64)

   
    c1, c2 = np.mean(landmarks_target, axis=0), np.mean(landmarks_img, axis=0)
    landmarks_target -= c1
    landmarks_img -= c2

 
    s1, s2 = np.std(landmarks_target), np.std(landmarks_img)
    landmarks_target /= s1
    landmarks_img /= s2

  
    U, S, Vt = np.linalg.svd(np.dot(landmarks_target.T, landmarks_img))
    R = (U * Vt).T

    
    matrix = np.vstack([
        np.hstack(((s2 / s1) * R, c2.T - (s2 / s1) * R * c1.T)),
        np.matrix([0., 0., 1.])
    ])

   
    result = np.zeros(shape_target, dtype=img.dtype)
    cv2.warpAffine(img,
                   matrix[:2], (shape_target[1], shape_target[0]),
                   dst=result,
                   borderMode=cv2.BORDER_TRANSPARENT,
                   flags=cv2.WARP_INVERSE_MAP)
    return result


def face_morph(bottom_img: np.ndarray,
               morph_img: np.ndarray,
               alpha: float = 0.5,
               exposure_alpha: float = 1.1) -> np.ndarray:
    
    landmarks_bottom = get_face_68_landmarks(bottom_img)

   
    morph_img = affine_transform_and_change_shape(morph_img, landmarks_bottom,
                                                  bottom_img.shape)

   
    landmarks_mask = get_face_68_landmarks(morph_img)

 
    morph_img = triangle_face_morph(bottom_img, morph_img, landmarks_bottom,
                                    landmarks_mask, float(alpha))

    merged_img = merge_img(bottom_img,
                           morph_img,
                           landmarks_bottom,
                           exposure_alpha=exposure_alpha)

   
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32)
    merged_img = cv2.filter2D(merged_img, -1, kernel=kernel)  
    merged_img = cv2.blur(merged_img, (3, 1)) 

    return merged_img




app.run(host='127.0.0.1', port='9000', debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:9000
Press CTRL+C to quit


5
6


127.0.0.1 - - [02/Dec/2023 16:19:28] "POST /upload_and_receive HTTP/1.1" 200 -


5
6


...

127.0.0.1 - - [02/Dec/2023 16:21:15] "POST /upload_and_receive HTTP/1.1" 200 -


5
6


127.0.0.1 - - [02/Dec/2023 16:21:49] "POST /upload_and_receive HTTP/1.1" 200 -


5
6


127.0.0.1 - - [02/Dec/2023 16:32:28] "POST /upload_and_receive HTTP/1.1" 200 -
